<a href="https://colab.research.google.com/github/wastb/Data-Warehouse-for-Ethiopian-Medical-Business/blob/master/notebooks/YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Overview***

*This section of the project focuses on using YOLOv5 to detect objects in images scraped from Telegram. The detection results, including bounding boxes, class labels, and confidence scores, will be structured and stored for seamless integration into the data warehouse.*

In [4]:
## Ensure you have the necessary dependencies installed
!pip install opencv-python
!pip install torch torchvision  # for PyTorch-based YOLO
!pip install tensorflow  # for TensorFlow-based YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [5]:
## Clone the YOLOv5 github repository
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17270, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17270 (delta 0), reused 0 (delta 0), pack-reused 17265 (from 1)
Receiving objects: 100% (17270/17270), 16.12 MiB | 13.39 MiB/s, done.
Resolving deltas: 100% (11858/11858), done.


In [6]:
## Navigate to the yolov5 folder
%cd yolov5

/content/yolov5


In [7]:
## Install the required packages
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 21.7 MB/s eta 0:00:00


In [8]:
## Detect objects in the images and save the result
!python detect.py --source /content/drive/MyDrive/photos --weights yolov5s.pt --save-txt --save-conf

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
detect: weights=['yolov5s.pt'], source=/content/drive/MyDrive/photos, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-398-g5cdad892 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)

100% 14.1M/14.1M [00:00<00:00, 172MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameter

#### ***Preprocess and Save the detection results***

In [10]:
import os
import csv

# Path to the labels folder
labels_folder = "/content/yolov5/runs/detect/exp/labels"
output_csv = "detection_results.csv"

# Open a CSV file for writing
with open(output_csv, mode="w", newline="") as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["Image", "Class ID", "Confidence", "x_center", "y_center", "width", "height"])

    # Iterate through all label files
    for label_file in os.listdir(labels_folder):
        if label_file.endswith(".txt"):
            image_name = label_file.replace(".txt", ".jpg")  # Assuming images are .jpg
            with open(os.path.join(labels_folder, label_file), "r") as f:
                lines = f.readlines()
                for line in lines:
                    class_id, x_center, y_center, width, height, confidence = map(float, line.split())
                    # Write a row for each detection
                    writer.writerow([image_name, int(class_id), confidence, x_center, y_center, width, height])

print(f"Results saved to {output_csv}")

Results saved to detection_results.csv


In [11]:
## View sample of the detection results
import pandas as pd

df = pd.read_csv("detection_results.csv")
df.head()

,Image,Class ID,Confidence,x_center,y_center,width,height
0,@EAHCI_2155.jpg,76,0.278768,0.669531,0.370313,0.198437,0.538542
1,@EAHCI_2155.jpg,0,0.439572,0.485547,0.499479,0.971094,0.984375
2,@DoctorsET_577.jpg,27,0.255653,0.039844,0.347222,0.023438,0.097222
3,@DoctorsET_577.jpg,56,0.316425,0.539062,0.520833,0.310937,0.372222
4,@DoctorsET_577.jpg,27,0.618466,0.034375,0.323611,0.017188,0.055556
